# How to use few shot examples
この Notebook は、LangChain の Models/Chat Models/How to use few shot examples で紹介されているコードを以下の条件で書き換えたものになります。
- Azure OpenAI Service を使用
- モデルは
    - gpt-4 (デプロイ名：gpt-4)

以下のリンクは、オリジナルのサイトになります。こちらも参照するようにしてください。   
https://python.langchain.com/en/latest/modules/models/chat/examples/few_shot_examples.html

### Environment Setup
必要なパラメータの値を、.env ファイルより取得します。あらかじめ.env ファイルに必要な値を設定しておいてください。

In [ ]:
import os
from dotenv import load_dotenv
import openai

# .env ファイルから環境変数を読み込む
load_dotenv("../../.env")

# モデルのデプロイ名をパラメータに設定
AZURE_OPENAI_GPT_DEPLOYMENT = os.environ.get("AZURE_OPENAI_GPT_DEPLOYMENT")    # text-davinchi-003 のデプロイ名
AZURE_OPENAI_GPT4_DEPLOYMENT = os.environ.get("AZURE_OPENAI_GPT4_DEPLOYMENT")  # gpt-4 のデプロイ名

# OpenAI API が使用するパラメータの設定
openai.api_type = "azure"
openai.api_base = os.environ.get("OPENAI_API_BASE")                            # Azure OpenAI API エンドポイント
openai.api_version = os.environ.get("OPENAI_API_VERSION")                      # Azure OpenAI API バージョン
openai.api_key = os.environ.get("OPENAI_API_KEY")                              # Azure OpenAI API キー

### Alternating Human/AI messages
Few-shot Prompting を行う最初の方法は、人と AI のメッセージを交互に交換することです。以下にその例を示します。

In [ ]:
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate
)

In [ ]:
chat = AzureChatOpenAI(deployment_name=AZURE_OPENAI_GPT4_DEPLOYMENT, temperature=0, openai_api_key=openai.api_key)

In [ ]:
template = "You are a helpful that translates English to Japanese."

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
example_human = HumanMessagePromptTemplate.from_template("Hi")
example_ai = AIMessagePromptTemplate.from_template("こんにちは")
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, example_human, example_ai, human_message_prompt])
chain = LLMChain(llm=chat, prompt=chat_prompt)

# フォーマットされたメッセージから Chat Completion を取得
chain.run("I love programming.")

### System Messages
OpenAI は、Few-shot prompting において、システムメッセージと組み合わせて使用できるオプションパラメータ `name` を提供しています。以下にその使用例を示します。

In [ ]:
template = "You are a helpful that translates English to pirate."

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
example_human = SystemMessagePromptTemplate.from_template("Hi", additional_kwargs={"name": "example_user"})
example_ai = SystemMessagePromptTemplate.from_template("Argh me mateys", additional_kwargs={"name": "example_assistant"})
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, example_human, example_ai, human_message_prompt])
chain = LLMChain(llm=chat, prompt=chat_prompt)

chain.run("I love programming.")